Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [2]:
NAME = "AVISHA BHIRYANI"

---

# Spherical k-Means Clustering

In this assignment, your task is to implement spherical k-means clustering *yourself*.

You will need to pay attention to performance. Using "for" loops over all instances and variables will not work, but instead you need to perform efficient vectorized operations.

In [3]:
import numpy as np, pandas as pd, scipy

In [4]:
# Load the input data
import json, gzip
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

Before you begin anything, always first have a look at the data you are dealing with!

In [5]:
classes

['Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Episodes',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Locations',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Episodes',
 'Episodes',
 'Characters',
 'Episodes',
 'Locations',
 'Episodes',
 'Characters',
 'Guest stars',
 'Characters',
 'Locations',
 'Characters',
 'Episodes',
 'Trivia',
 'Episodes',
 'Episodes',
 'Episodes',
 'Characters',
 'Episodes',
 'Episodes',
 'Guest stars',
 'Episodes',
 'Episodes',
 'Episodes',
 'Characters',
 'Characters',
 'Episodes',
 'Episodes',
 'Episodes',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Objects',
 'Characters',
 'Characters',
 'Characters',
 'Episodes',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 'Characters',
 

## Vectorize the text

Vectorize the Wiki texts, use the standard TF-IDF from the lecture (standard SMART `ltc` version, lowercase, *not* the scikit-learn variant) as discussed in the previous assignments. Use a minimum document frequency of 5 and standard english stopwords to reduce the vocabulary.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer # Please use this
vocab = None # Your vocabulary
dtm = None # Your sparse document term matrix
# YOUR CODE HERE
coun_vect = CountVectorizer(min_df=5, stop_words='english')
dtm = coun_vect.fit_transform(texts)
vocab = coun_vect.vocabulary_
print("Document term matrix has shape", dtm.shape)

Document term matrix has shape (10126, 14153)


In [7]:
def tf(dtm):
    """Compute the "l" step of standard TF-IDF"""
    # HINT: use dtm.astype(np.float32) to get a *sparse floating point copy* of the dtm matrix.
    dtm_float = dtm.astype(np.float32)
    #dtm_float_sum = dtm_float.sum(axis = 1)
    dtm_float_coo = scipy.sparse.coo_matrix(dtm_float)
    dtm_float_formean = scipy.sparse.coo_matrix(dtm_float)
    dtm_float_coo.data = np.log(dtm_float_coo.data)
    dtm_float_formean.data = dtm_float_coo.data
    dtm_float_coo.data = dtm_float_coo.data + 1
    
    dtm_float_formean_dense = dtm_float_formean.todense()
    dtm_float_formean_sparse =  scipy.sparse.csr_matrix(dtm_float_formean_dense)
    dtm_float_sum = dtm_float_formean_sparse.sum(axis=1)
    row,col = dtm_float_formean_dense.nonzero()
    for i in range(0,len(row)):
        dtm_float_formean_dense[row[i],col[i]] = dtm_float_formean_dense[row[i],col[i]] / dtm_float_sum[row[i]]

    dtm_float_log_f = scipy.sparse.csr_matrix(dtm_float_coo.todense())
    dtm_final = (dtm_float_log_f) / (1 + dtm_float_formean_dense)
    return scipy.sparse.csr_matrix(dtm_final)

In [8]:
import math
def idf(dtm):
    """ Compute the "t" step inverse document frequency """
    # YOUR CODE HERE
    dtm_float = dtm.astype(np.float32)
    dtm_float = scipy.sparse.csr_matrix(dtm_float)
    idfDict = {}
    i = 0
    for word in vocab:
        idfDict[word] = dtm_float.getcol(i).count_nonzero()
        i = i+1
    N = len(titles)
    arr = []
    #idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word, val in idfDict.items():
        arr.append(math.log(N / (float(val))))
    numpy_arr = np.array(arr)
    print(numpy_arr.max())
    return numpy_arr

In [9]:
def tfidf_func(dtm):
    """Finish the computation of standard TF-IDF with the c step"""
    _tf, _idf = tf(dtm), idf(dtm) # Must use above functions.
    # YOUR CODE HERE
    df = pd.DataFrame.sparse.from_spmatrix(_tf,index=titles,columns=vocab)
    i = 0
    for column in df.columns:
        df[column] = df[column] * _idf[i]
        df[column] = df[column].astype(np.float32)
        i = i+1
    csr_mat = scipy.sparse.csr_matrix(df.values)
    return csr_mat

In [10]:
tfidf = tfidf_func(dtm) # sparse tf-idf matrix
vocabulary = vocab # vocabulary
idf = idf(dtm) # IDF values
# YOUR CODE HERE


7.6134237400957545
7.6134237400957545


In [11]:
pd.DataFrame.sparse.from_spmatrix(tfidf, columns=vocabulary)

,good,night,simpsons,short,pilot,episode,televised,appearance,simpson,family,...,monticello,nora,lexington,concordia,bilderberger,ballmer,sandra,bullock,simpsonis,reenactor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.650708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### Automatic tests
assert tfidf is not None and vocabulary is not None, "Variables not set"
assert tfidf.shape[0] == len(texts), "Missing documents"
assert len(vocabulary) == tfidf.shape[1], "Vocabulary size does not match"
assert isinstance(tfidf, scipy.sparse.csr_matrix), "Not a sparse matrix"

In [13]:
### Automatic tests
assert len(idf) == tfidf.shape[1], "IDF size does not match"
assert idf.max() != 1 +np.log(len(texts)/5), "No, default sklearn is NOT okay"
assert idf.max() == np.log(len(texts)/5), "IDF does not match definition"
assert not isinstance(idf, scipy.sparse.csr_matrix), "IDF should not be sparse!"
assert isinstance(idf, np.ndarray), "IDF must be an array"

## Reassignment step

Implement the reassignment step of **spherical** k-means. Use **vectorized code**, or it will likely be too slow.

Do *not* use a Python `for` loop, and do *not* convert the input data to a dense matrix (slow).

In [14]:
from numpy import dot
from numpy.linalg import norm

def reassign(tfidf_in, centers):
    """Reassign each object in tfidf to the most similar center.
       Return a flat array, not a matrix."""
    # YOUR CODE HERE
    transposed_centers = np.transpose(centers)
    cos_sim = np.dot(tfidf_in, transposed_centers)
    max_sim_indices = cos_sim.argmax(axis=1)
    indices_reshaped = np.asarray(max_sim_indices).flatten()
    return indices_reshaped
# Test run
reassign(tfidf, tfidf[:5])

array([0, 1, 2, ..., 1, 0, 0])

In [15]:
### Automatic tests
_test = reassign(tfidf, tfidf[:5])
assert _test.shape == (tfidf.shape[0],), "Return shape does not match"
assert (_test[:5] == np.arange(0,5)).all(), "Incorrect results"
assert _test.min() == 0 and _test.max() == 4, "Invalid values in array"
assert isinstance(_test, np.ndarray), "Return value is not a dense array -- you may need to use asarray() on the return value, unfortunately."
assert _test.dtype == np.int64, "Not an integer array"
del _test

In [16]:
### Automatic tests
from unittest.mock import patch
with patch('__main__.range') as mock_r1, patch('numpy.arange') as mock_r2:
    reassign(tfidf[:10], tfidf[:5])
assert not mock_r1.called and not mock_r2.called, "Vectorize your code! Otherwise you will be waiting a long time below."
with patch('sklearn.metrics.pairwise.cosine_similarity') as mock_c1, patch('sklearn.metrics.pairwise.cosine_distances') as mock_c2:
    reassign(tfidf[:10], tfidf[:5])
assert not mock_c1.called and not mock_c2.called, "Use your own code, not sklearn."

## Recompute the cluster centers

Given a cluster assignment, recompute the cluster centers as used by *spherical* k-means.

Vectorize your code: do not iterate over all points with a Python for loop

Hint: for the assignment, it is okay to assume that a cluster never becomes empty.

In [17]:
def new_centers(tfidf_in, assignment):
    """Return a matrix containing the new cluster centers for spherical k-means."""
    centers = [] # Okay to use a list or an array for the assignment
    # YOUR CODE HERE
    for i in range(0,int(max(assignment))+1):
        cluster_points_index = np.flatnonzero((assignment == i))
        total_points = len(cluster_points_index)
        cluster_points = tfidf_in[cluster_points_index,:]
        
        mean_clutser = cluster_points.sum(axis=0) / total_points
        
        val = 1/np.sqrt(np.dot(mean_clutser, np.transpose(mean_clutser)).sum(1))
        normalized_mean_cluster = np.dot(mean_clutser,val[0,0])
        
        centers.append(normalized_mean_cluster)
    return np.array(centers) # Always return an array, copying is okay for the assignment

In [18]:
### Automatic tests
_tmp = new_centers(tfidf[:10], np.linspace(0, 4, 10).astype(np.int64))
assert len(_tmp) == 5, "Wrong number of centers."
for r in _tmp: assert r.shape[-1] == tfidf.shape[-1], "Not a proper center"
del _tmp

In [19]:
### Automatic tests
_tmp = new_centers(tfidf, np.zeros((tfidf.shape[0],)))
assert abs(np.array(_tmp).mean()-tfidf.mean()) > 1e-5, "This is not spherical k-means."
del _tmp

## Initialization

Now write initialization code. Given a random generator *seed*, chose `k` objects as initial cluster centers without replacement. Please use numpy.

In [20]:
import random
def initial_centers(tfidf_in, k, seed):
    """Choose k initial cluster centers."""
    # YOUR CODE HERE
    random.seed(seed)
    cluster_center_indices = []
    for _ in range(k):
        value = random.randint(0, tfidf_in.shape[0])
        cluster_center_indices.append(value)
    cluster_centers = tfidf_in[cluster_center_indices,:]
    return cluster_centers

In [21]:
### Automatic tests
_tmp = initial_centers(tfidf, 10, 42)
assert isinstance(_tmp, scipy.sparse.csr_matrix) or len(_tmp) == 10, "Wrong number of centers."
assert not isinstance(_tmp, scipy.sparse.csr_matrix) or _tmp.shape[0] == 10, "Wrong number of centers."
for r in _tmp: assert r.shape[-1] == tfidf.shape[-1], "Not a proper center."
del _tmp

In [22]:
### Automatic tests
assert (initial_centers(tfidf, 1, 42)-initial_centers(tfidf, 1, 42)).sum() == 0, "Seeding not okay."
assert (initial_centers(tfidf, 1, 42)-initial_centers(tfidf, 1, 21)).sum() != 0, "Seeding not okay."

## Implement a Quality Measure

As quality measure, compute the *sum* of cosine similarities of every point to its cluster center

In [23]:
from scipy import spatial
def quality(tfidf_in, centers, assignment):
    """Evaluate the quality given the current centers and cluster assignment."""
    # YOUR CODE HERE
    #tfidf_sum = tfidf.sum(axis=1)
    #normalized_tfidf = tfidf.multiply(scipy.sparse.csr_matrix(1/np.sqrt(tfidf.multiply(tfidf).sum(1))))
    #centers_sum = centers.sum(axis=1)
    #centers_transposed = np.transpose(centers)
    
    val = 1/np.sqrt(tfidf_in.multiply(tfidf_in).sum(1))
    normalized_tfidf = tfidf_in.multiply(val[0,0])
    s = []
    
    for i in range(0,int(max(assignment))+1):
        cluster_points_index = np.flatnonzero((assignment == i))

        cluster_points = normalized_tfidf[cluster_points_index,:]
        val1 = 1/np.sqrt(np.dot(centers[i], np.transpose(centers[i])).sum(1))
        normalized_cluster_center = np.dot(centers[i],val1[0,0])

        cos_sim = cluster_points.tocsr().multiply(normalized_cluster_center).sum(axis=1)
        s.append(cos_sim.sum())
    return sum(s)

In [24]:
### Automatic tests
# This test is likely slow if you use a "for" loop in quality(). But that is okay.
_tmp = quality(tfidf, tfidf[0], np.zeros((tfidf.shape[0],)))
assert quality(tfidf, tfidf, np.arange(0,tfidf.shape[0])) == tfidf.shape[0], "Result incorrect"
assert _tmp > 100, "This largely random result should score better"
assert _tmp < 500, "This largely random result should score less"

AssertionError: Result incorrect

As a reference value, compute the quality of assigning every object to the global *spherical* center.

Hint: you can use `new_centers` here.

In [25]:
center1 = None # Compute the overall center
sim1 = 0 # Compute the overall similarity

# YOUR CODE HERE
#not normalized
center1_val = tfidf.sum(axis=0) / tfidf.shape[0]
val = 1/np.sqrt(tfidf.multiply(tfidf).sum(1))

center1 = new_centers(tfidf, np.zeros((tfidf.shape[0],)))

normalized_tfidf = tfidf.multiply(val[0,0])
sim = normalized_tfidf.tocsr().multiply(center1).sum(axis=1)
sim1 = sim.sum()
print("Similarity sum to center:", sim1)
print("Average similarity to center:", sim1 / tfidf.shape[0])

Similarity sum to center: 1023.15784
Average similarity to center: 0.10104264634742866


In [26]:
### Automatic tests
assert center1 is not None, "Not answered."
assert abs(np.array(center1).mean()-tfidf.mean()) > 1e-5, "This is not the spherical center."
assert sim1 > 500, "This result should score better"
assert sim1 < 2000, "This result should score less"

## Implement Spherical k-Means

Now use these methods to implement spherical k-means clustering. Stop after a maximum number of iterations, or if no point is reassigned.

Return the cluster centers, the final cluster assignment, and an array of quality scores evaluated every time *after* reassigning the points to the clusters.

In [27]:
def spherical_kmeans(tfidf, init_centers, max_iter=100):
    qualities = []
    # YOUR CODE HERE
    k = 0
    if(type(init_centers) == int):
        k = init_centers
    else:
        k = init_centers.shape[0]
        
    for i in range(1,max_iter+1):
        centers = initial_centers(tfidf, k, 10)
        assignment = reassign(tfidf, centers)
        iter_quality = quality(tfidf, centers, assignment)

        if(i == 1):
            qualities.append(iter_quality)
        else:
            if(qualities[-1] < iter_quality):
                assignment = reassign(tfidf, centers)
                qualities.append(iter_quality)
            else:
                break
        centers = new_centers(tfidf, assignment)
            
    return centers.toarray(), assignment, qualities

In [28]:
### Automatic tests
from unittest.mock import patch
with patch('__main__.reassign') as mock_1, patch('__main__.new_centers') as mock_2, patch('__main__.quality') as mock_3:
    spherical_kmeans(tfidf, tfidf[0], 1)
    assert mock_1.called, "You did not use reassign"
    assert mock_2.called, "You did not use new_centers"
    assert mock_3.called, "You did not use quality"

## CLUSTER!

Now try out if your code works! First, cluster with `k=2`.

In [29]:
# YOUR CODE HERE
c,a,q = spherical_kmeans(tfidf, 2, max_iter=100)

In [30]:
### Automatic tests
_tmp = spherical_kmeans(tfidf, tfidf[[0,1]], 100)
assert len(_tmp[0]) == 2, "Wrong number of clusters"
assert _tmp[0].shape[-1] == tfidf.shape[-1], "Centers have bad shape"
assert sorted(np.unique(_tmp[1])) == [0,1], "Missing some clusters?"
assert len(_tmp[2]) < 90, "Should take much fewer iterations"
assert _tmp[2] == sorted(_tmp[2]), "Quality must be increasing"
assert _tmp[2][-1] == quality(tfidf, _tmp[0], _tmp[1]), "Quality wrong"
assert len(spherical_kmeans(tfidf, tfidf[[0,1]], 2)[2]) == 2, "max_iter incorrect."
del _tmp

AxisError: axis 1 is out of bounds for array of dimension 0

In [ ]:
### Automatic tests
_tmp = spherical_kmeans(tfidf, tfidf[[0,1]], 5)
_tmp2 = spherical_kmeans(tfidf, tfidf[[0,1]], 10)
assert _tmp[2] == _tmp2[2][:5]
del _tmp, _tmp2
# Additional hidden tests

## Study the Clusters

As we cannot rely on heuristics such as the "knee" to choose the number of clusters, we need to perform manual inspection:

- what are the most important words of each cluster?
- what are the most central documents in each cluster?

In [ ]:
def most_important(vocabulary, center, k=10):
    """Find the most important words for each cluster."""
    # YOUR CODE HERE
    #closest to the cluster center
    raise NotImplementedError()

In [ ]:
### Automatic tests
_tmp = tfidf[0].toarray()[0]
assert len(most_important(vocabulary, _tmp, 42)) == 42, "Wrong number of results."
for x in most_important(vocabulary, _tmp): assert isinstance(x, str), "Not words."

In [ ]:
def most_central(tfidf, centers, assignment, i, k=5):
    """Find the most central documents of cluster i"""
    # YOUR CODE HERE
    #closest to the cluster center
    raise NotImplementedError()

In [ ]:
### Automatic tests
assert len(most_central(tfidf, tfidf[[0]].toarray(), np.zeros((tfidf.shape[0],)), 0, 42)) == 42, "Wrong number of results."
assert (most_central(tfidf, tfidf[[0,1]].toarray(), np.arange(0,tfidf.shape[0])&1, 0, 10)&1==0).all(), "Only documents from the same cluster may be returned."

## Explain your Clusters

Write a function to print a cluster explanation using above functions, and run it for k=20.

In [ ]:
def explain(tfidf, vocabulary, titles, centers, assignment):
    """Use what you built."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Cluster with k=20, and explain!
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
### Automatic tests
with patch('__main__.most_important') as mock_1, patch('__main__.most_central') as mock_2, patch('__main__.print') as mock_3:
    explain(tfidf, vocabulary, titles, tfidf[[0,1]].toarray(), np.arange(0,tfidf.shape[0])&1)
    assert mock_1.called, "You did not use most_important"
    assert mock_2.called, "You did not use most_central"
    assert mock_3.called, "You did not print"